# Importing the required packages

In [1]:
%matplotlib notebook
import numpy as np
import cvxopt
import cvxopt.glpk

In [2]:
cvxopt.glpk.ilp?

In [3]:
## To help create the huge matrix

#  If the condition is true, it does nothing and your program just continues to execute. 
#But if the assert condition evaluates to false, it raises an AssertionError exception with an optional error message.
def unravel(i,j,k,size=3):
    n2 = size*size
    assert(i>=0 and i<n2)
    assert(j>=0 and i<n2)
    assert(k>=0 and i<n2)
    
    return(k+ j*n2+ i*n2*n2)


""" not used
def ravel(l,size=3):
    n2=size*size
    assert (l>=0 and l < n2*n2*n2)
    i = l // (n2*n2)
    j = (l % (n2*n2)) // n2
    k = l - i*n2*n2 - j*n2
    return ((i,j,k))
"""

' not used\ndef ravel(l,size=3):\n    n2=size*size\n    assert (l>=0 and l < n2*n2*n2)\n    i = l // (n2*n2)\n    j = (l % (n2*n2)) // n2\n    k = l - i*n2*n2 - j*n2\n    return ((i,j,k))\n'

In [4]:
## create constraint matrix

size=3
n2=size*size
A=np.zeros((4*n2*n2,n2*n2*n2))
A.shape

(324, 729)

In [5]:
## line constraints: only one number per line
c=0
for k in range(n2): ## for all numbers
    for j in range(n2): ## for all columns
        for i in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number k on line i
            A[c+n2*n2,unravel(j,i,k)] = 1 ##only one number k on line j
            A[c+2*n2*n2,unravel(j,k,i)] = 1 ## number k only once in the entire table
            f=0                             ## number k only once in a subsquare
            for m in range(size):
                for n in range(size):
                    A[c//n2+3*n2*n2+n2*f,unravel((i//size + size*m) , (j//size +size*n) ,k)] = 1
                    f+=1
        c += 1
           
        
print("Total number of constraints=",c)

def testA(A,c,size=3):
    n2 = size*size
    for n in range(c):
        if (np.sum(A[n,])!=n2):
            print("error on line", n)
            break
    print("All constraints OK")
    return

testA(A,c)

Total number of constraints= 81
All constraints OK


In [6]:
## add the fixed numbers constraints

# important to note - the indices range from 0 to 8 and not from 1 to 9

ls = [(0,1,7),(0,3,8),(0,5,0),(0,7,4),(1,2,1),(1,3,5),(1,4,7),(1,5,6),(1,6,2),(2,2,2),(2,6,5),(3,0,2),
     (3,1,8),(3,7,5),(3,8,4),(4,0,5),(4,3,3),(4,4,6),(4,5,4),(4,8,2),(5,0,4),(5,1,6),(5,7,7),(5,8,3),(6,2,8),(6,6,7),(7,2,4),
      (7,3,0),(7,4,1),(7,5,3),(7,6,8),(8,1,3),(8,3,7),(8,5,2),(8,7,1)]
for lenconst in range(len(ls)):
    i,j,k = ls[lenconst]
    new = np.zeros((1,A.shape[1]))
    new[0,unravel(i,j,k)] = 1
    A = np.append(A,new,axis=0)
            
print("A.shape=", A.shape)

A.shape= (359, 729)


In [7]:
## solving
from cvxopt import matrix

b=matrix(np.ones(A.shape[0])) ## set partition
c=matrix(np.zeros(A.shape[1])) ## zero cost
G=matrix(np.zeros(A.shape))
h=matrix(np.zeros(A.shape[0]))
binary=np.array(range(A.shape[1]))
I=set(binary)
B=set(binary)
Aeq = matrix(A)
(status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))

In [8]:
(status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))

In [9]:
status

'optimal'

# Getting final output

In [10]:
## print solution
def printsol(sol):
    sep3="+-----+-----+-----+"
    for i in range(n2):
        if (i%3 == 0):
            print(sep3)
        print("|",end='')
        for j in range(n2):
            for k in range(n2):
                if (sol[unravel(i,j,k)]==1):
                    print(k+1,end='')
            if (j%3 ==2):
                print("|",end='')
            else:
                print(" ",end='')
        print("")
    print(sep3)
        
printsol(solution)
          

+-----+-----+-----+
|7 8 6|9 3 1|4 5 2|
|4 5 2|6 8 7|3 1 9|
|9 1 3|5 4 2|6 7 8|
+-----+-----+-----+
|3 9 4|2 1 8|7 6 5|
|6 2 8|4 7 5|1 9 3|
|5 7 1|3 6 9|2 8 4|
+-----+-----+-----+
|2 3 9|7 5 6|8 4 1|
|8 6 5|1 2 4|9 3 7|
|1 4 7|8 9 3|5 2 6|
+-----+-----+-----+
